## torch.Tensor.register_hook


In [2]:
import torch
y_grad = list()


def grad_hook(grad):
    y_grad.append(grad)


x = torch.tensor([2., 2., 2., 2.], requires_grad=True)
y = torch.pow(x, 2)
z = torch.mean(y)
# 这里 register_hook 注册了一个钩子，当 y 的梯度计算完成后，梯度值会传递给 grad_hook 并存储到 y_grad 中
h = y.register_hook(grad_hook)
z.backward()
# y.grad 为 None，因为 y 不是叶子节点，它是 x 计算得到的中间变量。
# 但 y_grad[0] 存储了钩子捕获的梯度，即 [0.25, 0.25, 0.25, 0.25]。
print("y.grad: ", y.grad)
print("y_grad[0]: ", y_grad[0])
print("z.grad: ", z.grad)
# # 移除 hook
h.remove()    # removes the hook

y.grad:  None
y_grad[0]:  tensor([0.2500, 0.2500, 0.2500, 0.2500])
z.grad:  None


C:\Users\admin\AppData\Local\Temp\ipykernel_3908\1787383474.py:17: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print("y.grad: ", y.grad)
C:\Users\admin\AppData\Local\Temp\ipykernel_3908\1787383474.py:19: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tens

可以看到当 z.backward()结束后，张量 y 中的 grad 为 None，因为 y 是非叶子节点张量，在梯度反传结束之后，被释放。  
在对张量 y 的 hook 函数（grad_hook）中，将 y 的梯度保存到了 y_grad 列表中，因此可以在 z.backward()结束后，仍旧可以在 y_grad[0]中读到 y 的梯度为 tensor([0.2500, 0.2500, 0.2500, 0.2500])


In [6]:
import torch


def grad_hook(grad):
    grad *= 2


x = torch.tensor([2., 2., 2., 2.], requires_grad=True)
y = torch.pow(x, 2)
z = torch.mean(y)
h = x.register_hook(grad_hook)
z.backward()
print(x.grad)
h.remove()    # removes the hook

tensor([2., 2., 2., 2.])


原 x 的梯度为 tensor([1., 1., 1., 1.])，经 grad_hook 操作后，梯度为 tensor([2., 2., 2., 2.])。


## torch.nn.Module.register_forward_hook


In [ ]:
import torch
import torch.nn as nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)  # 输入通道 1，输出通道 2，3x3 卷积核
        self.pool1 = nn.MaxPool2d(2, 2)  # 2x2 最大池化

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x
# Hook 作用： 记录中间层数据，方便可视化特征图。


def farward_hook(module, data_input, data_output):
    fmap_block.append(data_output)  # 保存输出特征图
    input_block.append(data_input)  # 保存输入数据


# 初始化网络
# conv1.weight[0]：第一组 3x3 卷积核，全部填充为 1。
# conv1.weight[1]：第二组 3x3 卷积核，全部填充为 2。
# conv1.bias.data.zero_()：设置 bias=0，避免影响结果。
net = Net()
net.conv1.weight[0].detach().fill_(1)
net.conv1.weight[1].detach().fill_(2)
net.conv1.bias.data.zero_()
# 注册hook
fmap_block = list()
input_block = list()
# register_forward_hook 绑定 conv1，执行时会调用 farward_hook。
net.conv1.register_forward_hook(farward_hook)
# inference
fake_img = torch.ones((1, 1, 4, 4))   # batch size * channel * H * W
output = net(fake_img)
# 观察
print("output shape: {}\noutput value: {}\n".format(output.shape, output))
print("feature maps shape: {}\noutput value: {}\n".format(
    fmap_block[0].shape, fmap_block[0]))
print("input shape: {}\ninput value: {}".format(
    input_block[0][0].shape, input_block[0]))

output shape: torch.Size([1, 2, 1, 1])
output value: tensor([[[[ 9.]],

         [[18.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)

feature maps shape: torch.Size([1, 2, 2, 2])
output value: tensor([[[[ 9.,  9.],
          [ 9.,  9.]],

         [[18., 18.],
          [18., 18.]]]], grad_fn=<ConvolutionBackward0>)

input shape: torch.Size([1, 1, 4, 4])
input value: (tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)


首先初始化一个网络，卷积层有两个卷积核，权值分别为全 1 和全 2，bias 设置为 0，池化层采用 2\*2 的最大池化。

在进行 forward 之前对 module——conv1 注册了 forward_hook 函数，然后执行前向传播（output=net(fake_img)），当前向传播完成后， fmap_block 列表中的第一个元素就是 conv1 层输出的特征图了。

这里注意观察 farward_hook 函数有 data_input 和 data_output 两个变量，特征图是 data_output 这个变量，而 data_input 是 conv1 层的输入数据， conv1 层的输入是一个 tuple 的形式。
